In [341]:
# binary classification, breast cancer dataset, label and one hot encoded
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder



In [342]:
# load data
# load data
filepaht="datasets\S50F_DataSets\ML_S50IF_CON_2014-2019-TrainData.csv"
data =pd.read_csv(filepaht,index_col="datetime", parse_dates=['datetime'],dayfirst=True)
print(data.info())
data.tail(3)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1465 entries, 2014-01-02 09:45:00 to 2019-12-30 09:45:00
Data columns (total 18 columns):
ma-20             1465 non-null float64
ma-50             1465 non-null float64
ma-75             1465 non-null float64
trend-20          1465 non-null int64
trend-50          1465 non-null int64
trend-75          1465 non-null int64
hh-50             1465 non-null float64
ll-50             1465 non-null float64
mid50             1465 non-null float64
rannkHL50         1465 non-null int64
macd5-20          1465 non-null float64
signal5-20-5      1465 non-null float64
fast_hist5-20     1465 non-null float64
rsi-10            1465 non-null float64
MACDTrend         1465 non-null int64
RsiTrend          1465 non-null int64
MACD_RSI_Trend    1465 non-null int64
TrendLabel        1465 non-null int64
dtypes: float64(10), int64(8)
memory usage: 217.5 KB
None


,ma-20,ma-50,ma-75,trend-20,trend-50,trend-75,hh-50,ll-50,mid50,rannkHL50,macd5-20,signal5-20-5,fast_hist5-20,rsi-10,MACDTrend,RsiTrend,MACD_RSI_Trend,TrendLabel
datetime,,,,,,,,,,,,,,,,,,
2019-12-26 09:45:00,1062.33,1074.89,1078.61,1,0,0,1091.5,1040.5,1066.0,2,6.39,1.61,4.78,56.17,4,3,4,2
2019-12-27 09:45:00,1061.13,1074.49,1078.04,1,0,0,1091.5,1040.5,1066.0,2,7.28,4.17,3.10,47.95,4,2,3,2
2019-12-30 09:45:00,1060.48,1074.14,1077.55,1,0,0,1091.5,1040.5,1066.0,2,7.28,5.77,1.51,50.90,4,3,4,2


In [343]:
#data.isnull().sum()
categorical_columns=['trend-20','trend-50','trend-75','rannkHL50','MACDTrend','RsiTrend','MACD_RSI_Trend']
number_columns=[  x  for x in  data.columns if (x not in categorical_data) and x !='TrendLabel' ]
print(categorical_data)
print(number_columns)

['trend-20', 'trend-50', 'trend-75', 'rannkHL50', 'MACDTrend', 'RsiTrend', 'MACD_RSI_Trend']
['ma-20', 'ma-50', 'ma-75', 'hh-50', 'll-50', 'mid50', 'macd5-20', 'signal5-20-5', 'fast_hist5-20', 'rsi-10']


In [344]:
normical_data= data[categorical_columns]
numberic_data=data[number_columns]


In [345]:
normical_data.head()

,trend-20,trend-50,trend-75,rannkHL50,MACDTrend,RsiTrend,MACD_RSI_Trend
datetime,,,,,,,
2014-01-02 09:45:00,1,0,1,3,1,1,1
2014-01-03 09:45:00,1,0,1,3,1,1,1
2014-01-06 09:45:00,1,0,0,3,1,1,1
2014-01-07 09:45:00,1,0,0,3,1,1,1
2014-01-08 09:45:00,1,0,0,3,1,1,1


In [346]:
numberic_data.head()

,ma-20,ma-50,ma-75,hh-50,ll-50,mid50,macd5-20,signal5-20-5,fast_hist5-20,rsi-10
datetime,,,,,,,,,,
2014-01-02 09:45:00,911.91,945.41,956.83,951.8,869.3,910.55,-37.25,-21.21,-16.04,12.93
2014-01-03 09:45:00,906.27,941.76,955.11,951.4,815.8,883.60,-48.27,-28.72,-19.55,15.97
2014-01-06 09:45:00,900.83,938.03,953.48,951.4,796.8,874.10,-57.09,-37.39,-19.70,22.14
2014-01-07 09:45:00,896.10,935.31,951.68,951.4,796.8,874.10,-57.66,-45.07,-12.58,35.93
2014-01-08 09:45:00,891.59,932.43,949.87,951.4,796.8,874.10,-57.65,-51.58,-6.06,34.93


In [347]:
# list uniqu value excluding Label
for col in categorical_columns:
 print(data[col].unique())


[1 2 3]
[0 1]
[1 0]
[3 2 1]
[1 2 4 3]
[1 2 3 4]
[1 2 3 4]


# Encode string input values as integers

In [348]:

X_norminal = normical_data.to_numpy()
X_numeric=numberic_data.to_numpy()

print(X_norminal[:5,:])
print(X_numeric[:5,:])


[[1 0 1 3 1 1 1]
 [1 0 1 3 1 1 1]
 [1 0 0 3 1 1 1]
 [1 0 0 3 1 1 1]
 [1 0 0 3 1 1 1]]
[[911.91 945.41 956.83 951.8  869.3  910.55 -37.25 -21.21 -16.04  12.93]
 [906.27 941.76 955.11 951.4  815.8  883.6  -48.27 -28.72 -19.55  15.97]
 [900.83 938.03 953.48 951.4  796.8  874.1  -57.09 -37.39 -19.7   22.14]
 [896.1  935.31 951.68 951.4  796.8  874.1  -57.66 -45.07 -12.58  35.93]
 [891.59 932.43 949.87 951.4  796.8  874.1  -57.65 -51.58  -6.06  34.93]]


In [349]:
Y = (data['TrendLabel']).values
print(Y.shape)
Y[:5]

(1465,)


array([2, 2, 2, 2, 2], dtype=int64)

In [350]:
columns = []
for i in range(0, X_norminal.shape[1]):
#for i in range(0, X.shape[1]):
 print("Col: ",i,"\n=======================================")
    
 label_encoder = LabelEncoder()
 feature = label_encoder.fit_transform(X_norminal[:,i])
 print("fit_transform")
 print(feature)

 feature = feature.reshape(X_norminal.shape[0], 1)
 #print("resharp  to 2 dim")
 #print(feature)

 onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
 feature = onehot_encoder.fit_transform(feature)
 print("onehot_encoder as no of category")
 print(feature)

 columns.append(feature)
    

#print(columns)


Col:  0 
fit_transform
[0 0 0 ... 0 0 0]
onehot_encoder as no of category
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]
Col:  1 
fit_transform
[0 0 0 ... 0 0 0]
onehot_encoder as no of category
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Col:  2 
fit_transform
[1 1 0 ... 0 0 0]
onehot_encoder as no of category
[[0. 1.]
 [0. 1.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Col:  3 
fit_transform
[2 2 2 ... 1 1 1]
onehot_encoder as no of category
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]
Col:  4 
fit_transform
[0 0 0 ... 3 3 3]
onehot_encoder as no of category
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]
Col:  5 
fit_transform
[0 0 0 ... 2 1 2]
onehot_encoder as no of category
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
Col:  6 
fit_transform
[0 0 0 ... 3 2 3]
onehot_encoder as no of category
[[1. 0. 0. 0.]

In [351]:
# collapse columns into array
encoded_X_norminal = numpy.column_stack(columns)
print("X_norminal shape: : ",encoded_X_norminal.shape)

X_norminal shape: :  (1465, 22)


In [352]:

# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_Y = label_encoder.transform(Y)

print(label_encoded_Y.shape)
label_encoded_Y

(1465,)


array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [353]:
print(encoded_X_norminal.shape)
encoded_X_norminal[:3,:]


(1465, 22)


array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 0., 1., 0., 0., 0.]])

In [354]:
print(X_numeric.shape)
X_numeric[:3,:]

(1465, 10)


array([[911.91, 945.41, 956.83, 951.8 , 869.3 , 910.55, -37.25, -21.21,
        -16.04,  12.93],
       [906.27, 941.76, 955.11, 951.4 , 815.8 , 883.6 , -48.27, -28.72,
        -19.55,  15.97],
       [900.83, 938.03, 953.48, 951.4 , 796.8 , 874.1 , -57.09, -37.39,
        -19.7 ,  22.14]])

In [355]:
X=np.concatenate((encoded_X_norminal, X_numeric), axis=1)
print(X.shape)
X[:3,:]

(1465, 32)


array([[  1.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   1.  ,   1.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  , 911.91, 945.41,
        956.83, 951.8 , 869.3 , 910.55, -37.25, -21.21, -16.04,  12.93],
       [  1.  ,   0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   1.  ,   1.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  , 906.27, 941.76,
        955.11, 951.4 , 815.8 , 883.6 , -48.27, -28.72, -19.55,  15.97],
       [  1.  ,   0.  ,   0.  ,   1.  ,   0.  ,   1.  ,   0.  ,   0.  ,
          0.  ,   1.  ,   1.  ,   0.  ,   0.  ,   0.  ,   1.  ,   0.  ,
          0.  ,   0.  ,   1.  ,   0.  ,   0.  ,   0.  , 900.83, 938.03,
        953.48, 951.4 , 796.8 , 874.1 , -57.09, -37.39, -19.7 ,  22.14]])

In [356]:
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, label_encoded_Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)
print(model)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)
Accuracy: 85.12%
